In [7]:
import random
import numpy as np

In [8]:
def make_diag_mask(size, radius):
    """Квадратная матрица размера Size x Size с двумя полосами ширины radius вдоль главной диагонали"""
    #idxs = torch.arange(size)
    #abs_idx_diff = (idxs.unsqueeze(0) - idxs.unsqueeze(1)).abs()
    #mask = ((abs_idx_diff <= radius) & (abs_idx_diff > 0)).float()
    mask = np.ones((size, size))
    for i in range(size):
        for j in range(size): 
            if ((i == j) | (i - j > radius) | (j - i > radius)):
                mask[i, j] = 0
    return mask

In [9]:
def generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords):
    # преобразование token2subwords с учетом текста    
    for n,i in enumerate(token2subwords):
        if n in text:
            token2subwords[n].append(n)
            tmp_dict = {i for i in token2subwords[n]}
            token2subwords[n] = sorted([i for i in tmp_dict])
    # удаление пустых токенов
    for i in token2subwords:
        if i == []:
            token2subwords.remove(i)
    # создание матрицы-маски
    matrix_mask = make_diag_mask(len(text), window_size // 2)
    # создание списка положительных примеров
    positives = []
    for i,row in zip(text,matrix_mask):
        for j,m in zip(text,row):
            if m == 1:
                positives.append([i,j,1])
    # преобразование списка положительных примеров
    for i in range(len(positives)):
        for j in token2subwords:
            if positives[i][0] == j[0]:
                positives[i][0] = j
    # создание списка случайных чисел для создания отрицательных примеров
    rands = [random.randint(vocab_size+1, 100) for i in range(1000)]
    for i in text:
        if i in rands:
            rands.remove(i)
    # создание списка отрицательных примеров
    negatives = []   
    for i in text:
        for j in rands:
            negatives.append([i,j,0])
    # преобразование списка отрицательных примеров
    for i in range(len(negatives)):
        for j in token2subwords:
            negatives[i][0] = j 
    #определение количества необходимых отрицательных примеров
    num_of_negatives = len(positives) * ns_rate
    # объединение положительных и отрицательных примеров
    for i in negatives[:num_of_negatives]:
        positives.append(i)
    return positives

In [10]:
generate_ft_sgns_samples([1, 2, 0, 1, 4, 0, 4, 1, 5, 4, 5, 4, 5, 1], 3, 6, 2, [[17], [10, 12], [20, 20], [7, 13], [], [7, 11]])

[[[1, 10, 12], 2, 1],
 [[1, 10, 12], 0, 1],
 [[1, 10, 12], 1, 1],
 [[2, 20], 1, 1],
 [[2, 20], 0, 1],
 [[2, 20], 1, 1],
 [[2, 20], 4, 1],
 [[0, 17], 1, 1],
 [[0, 17], 2, 1],
 [[0, 17], 1, 1],
 [[0, 17], 4, 1],
 [[0, 17], 0, 1],
 [[1, 10, 12], 1, 1],
 [[1, 10, 12], 2, 1],
 [[1, 10, 12], 0, 1],
 [[1, 10, 12], 4, 1],
 [[1, 10, 12], 0, 1],
 [[1, 10, 12], 4, 1],
 [[4], 2, 1],
 [[4], 0, 1],
 [[4], 1, 1],
 [[4], 0, 1],
 [[4], 4, 1],
 [[4], 1, 1],
 [[0, 17], 0, 1],
 [[0, 17], 1, 1],
 [[0, 17], 4, 1],
 [[0, 17], 4, 1],
 [[0, 17], 1, 1],
 [[0, 17], 5, 1],
 [[4], 1, 1],
 [[4], 4, 1],
 [[4], 0, 1],
 [[4], 1, 1],
 [[4], 5, 1],
 [[4], 4, 1],
 [[1, 10, 12], 4, 1],
 [[1, 10, 12], 0, 1],
 [[1, 10, 12], 4, 1],
 [[1, 10, 12], 5, 1],
 [[1, 10, 12], 4, 1],
 [[1, 10, 12], 5, 1],
 [[5, 7, 11], 0, 1],
 [[5, 7, 11], 4, 1],
 [[5, 7, 11], 1, 1],
 [[5, 7, 11], 4, 1],
 [[5, 7, 11], 5, 1],
 [[5, 7, 11], 4, 1],
 [[4], 4, 1],
 [[4], 1, 1],
 [[4], 5, 1],
 [[4], 5, 1],
 [[4], 4, 1],
 [[4], 5, 1],
 [[5, 7, 11], 1, 1],
 